In [ ]:
import torch
from torch import nn
import numpy as np
from myConv import myConv2d

In [ ]:
# import data and convert to tensor
import tifffile as tiff
pavia = tiff.imread(r'E:\Hyperspectral Scenes\Pavia Center and University_down\pavia.tif')
pavia_gt = tiff.imread(r'E:\Hyperspectral Scenes\Pavia Center and University_down\pavia_gt.tif')
pavia = np.moveaxis(np.array(pavia, dtype=int), -1, 0)
pavia_gt = np.array(pavia_gt, dtype=int)
X = torch.from_numpy(pavia)
y = torch.from_numpy(pavia_gt)
X.shape, y.shape

In [ ]:
from torch.utils.data import DataLoader, Dataset

# split data into train and test
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# dataset class
class HyperDataset(Dataset):
    def __init__(self, image_path, gt_path):
        self.image = tiff.imread(image_path)
        self.image = np.moveaxis(np.array(self.image, dtype=np.float32), -1, 0)
        self.image = torch.from_numpy(self.image)
        print(self.image.dtype)

        self.gt = tiff.imread(gt_path)
        self.gt = np.array(self.gt, dtype=np.float32)
        self.gt = torch.from_numpy(self.gt)
        self.indices = np.nonzero(self.gt[5:-5, 5:-5])
        self.nonzero = np.count_nonzero(self.gt[5:-5, 5:-5])

    def __len__(self):
        return self.nonzero

    def __getitem__(self, index):
        x = self.indices[index][0] + 5
        y = self.indices[index][1] + 5
        image = self.image[:, x-5:x+5, y-5:y+5]
        gt = self.gt[x, y]
        return image, gt

dataset = HyperDataset(r'E:\Hyperspectral Scenes\Pavia Center and University_down\pavia.tif',
                       r'E:\Hyperspectral Scenes\Pavia Center and University_down\pavia_gt.tif')
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [ ]:
from torch import optim
from torch.nn.functional import relu

lr = 0.001

# define net
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # self.conv1 = myConv2d(102, 64, kernel_size=[3, 3], stride=1)
        self.conv1 = nn.Conv2d(102, 64, 3, 1, 1)
        # self.conv2 = myConv2d(64, 32, kernel_size=[3, 3], stride=1)
        self.conv2 = nn.Conv2d(64, 32, 3, 1, 1)
        # self.conv3 = myConv2d(32, 16, kernel_size=[3, 3], stride=1)
        self.conv3 = nn.Conv2d(32, 16, 3, 1, 1)
        self.fc = nn.Linear(16*10*10, 10)

    def forward(self, x):
        # x = x.float()
        out = relu(self.conv1(x))
        out = relu(self.conv2(out))
        out = relu(self.conv3(out))
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

In [ ]:
# train
for epoch in range(10):
    for i, (data, labels) in enumerate(train_loader):
        # forward
        outputs = model(data)
        loss = criterion(outputs, labels.long())

        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print analysis
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, 10, i+1, len(train_loader), loss.item()))
        if i > 10:
            break

In [ ]:
# test
with torch.no_grad():
    correct = 0
    total = 0
    for data, labels in test_loader:
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.squeeze()).sum().item()

    print('Test Accuracy of the model on the test data: {} %'.format(100 * correct / total))